In [1]:
! docker run --name turminhadb -d -e MYSQL_ROOT_PASSWORD=123456 -e MYSQL_DATABASE=turminha -p 8081:3306 --rm  mysql:8.1.0

8bc11ad647c1f0e380d66d532daf4e08212ce73976c9468becb2fb5afa91cb1f


In [2]:
! sleep 20s

In [3]:
! pip install pymysql

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import pymysql

In [5]:
def conexao():
    try:
        conn = pymysql.connect(
            user='root',
            host='127.0.0.1',
            port=8081,
            password='123456',
            database = "turminha",
        )
        return conn
    except pymysql.Error as e:
        raise Exception('Erro ao se conectar: ' + str(e))

In [6]:
connection = conexao()

In [7]:
# Função para criar a tabela aluno
def criar_tabela_aluno(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS aluno (
                id_aluno INT AUTO_INCREMENT PRIMARY KEY,
                nome_aluno VARCHAR(255),
                aluno_alocado BOOLEAN,
                id_turma INT
            )
        """)
        connection.commit()
        print("Tabela aluno criada com sucesso.")
    except pymysql.Error as e:
        print("Erro ao criar tabela aluno:", str(e))



In [8]:
def criar_tabela_turma(connection):
    cursor = connection.cursor()

    try:
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS turma (
                id_turma INT AUTO_INCREMENT PRIMARY KEY,
                codigo_turma VARCHAR(255),
                nome_turma VARCHAR(255)
            )
        """)
        connection.commit()
        print("Tabela 'turma' criada com sucesso.")
    except pymysql.Error as e:
        print('Erro ao criar tabela turma:', e)

In [9]:
# Criar as tabelas
criar_tabela_aluno(connection)
criar_tabela_turma(connection)

Tabela aluno criada com sucesso.
Tabela 'turma' criada com sucesso.


a) Inserir pelo menos duas turmas diferentes na tabela de turma:

In [10]:
def inserir_turmas(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("INSERT INTO turma (codigo_turma, nome_turma) VALUES (%s, %s)", ("T1", "Manuel Bandeira"))
        cursor.execute("INSERT INTO turma (codigo_turma, nome_turma) VALUES (%s, %s)", ("T2", "Fernando Pessoa"))
        connection.commit()
        print("Turmas inseridas com sucesso.")
    except pymysql.Error as e:
        print("Erro ao inserir turmas:", str(e))



In [11]:
# Chamando a função para inserir turmas
inserir_turmas(connection)

Turmas inseridas com sucesso.


b) Inserir pelo menos 1 aluno alocado em cada uma destas turmas na tabela aluno (todos com NULL na coluna aluno_alocado):

In [12]:
# Função para inserir alunos
def inserir_alunos(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("INSERT INTO aluno (nome_aluno, aluno_alocado, id_turma) VALUES (%s, NULL, %s)", ("Clara Ferreira Batista", 1))
        cursor.execute("INSERT INTO aluno (nome_aluno, aluno_alocado, id_turma) VALUES (%s, NULL, %s)", ("Carlos Eduardo Abade", 2))
        connection.commit()
        print("Alunos inseridos com sucesso.")
    except pymysql.Error as e:
        print("Erro ao inserir alunos:", str(e))



# Chamando a função para inserir alunos
inserir_alunos(connection)


Alunos inseridos com sucesso.


c) Inserir pelo menos 2 alunos não alocados em nenhuma turma na tabela aluno (todos com NULL na coluna aluno_alocado):

In [13]:
# Função para inserir alunos não alocados
def inserir_alunos_nao_alocados(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("INSERT INTO aluno (nome_aluno, aluno_alocado) VALUES (%s, NULL)", ("Charllote Ferreira Batista",))
        cursor.execute("INSERT INTO aluno (nome_aluno, aluno_alocado) VALUES (%s, NULL)", ("Bruce Abade",))
        connection.commit()
        print("Alunos não alocados inseridos com sucesso.")
    except pymysql.Error as e:
        print("Erro ao inserir alunos não alocados:", str(e))

inserir_alunos_nao_alocados(connection)



Alunos não alocados inseridos com sucesso.


d) Atualizar a coluna aluno_alocado da tabela aluno:

In [14]:
# Função para atualizar o status de aluno alocado
def atualizar_aluno_alocado(connection, aluno_id, alocado):
    try:
        cursor = connection.cursor()
        cursor.execute("UPDATE aluno SET aluno_alocado = %s WHERE id_aluno = %s", (alocado, aluno_id))
        connection.commit()
        print(f"Aluno {aluno_id} atualizado para alocado = {alocado}.")
    except pymysql.Error as e:
        print("Erro ao atualizar aluno:", str(e))

# Atualizar alunos alocados
atualizar_aluno_alocado(connection, 1, True)
atualizar_aluno_alocado(connection, 2, True)

# Atualizar alunos não alocados
atualizar_aluno_alocado(connection, 3, False)
atualizar_aluno_alocado(connection, 4, False)


Aluno 1 atualizado para alocado = True.
Aluno 2 atualizado para alocado = True.
Aluno 3 atualizado para alocado = False.
Aluno 4 atualizado para alocado = False.


In [15]:
def consultar_turmas_e_alunos(connection):
    try:
        cursor = connection.cursor()

        # Consulta para buscar todas as turmas
        cursor.execute("SELECT * FROM turma")
        turmas = cursor.fetchall()

        # Consulta para buscar todos os alunos
        cursor.execute("SELECT * FROM aluno")
        alunos = cursor.fetchall()

        # Imprimir as turmas
        print("TABELA turma:")
        for turma in turmas:
            print(turma)

        # Imprimir os alunos
        print("\nTABELA aluno:")
        for aluno in alunos:
            print(aluno)

    except pymysql.Error as e:
        print("Erro ao consultar turmas e alunos:", str(e))

consultar_turmas_e_alunos(connection)


TABELA turma:
(1, 'T1', 'Manuel Bandeira')
(2, 'T2', 'Fernando Pessoa')

TABELA aluno:
(1, 'Clara Ferreira Batista', 1, 1)
(2, 'Carlos Eduardo Abade', 1, 2)
(3, 'Charllote Ferreira Batista', 0, None)
(4, 'Bruice Abade', 0, None)


In [16]:
! docker container stop turminhadb

turminhadb
